## Test book content loader

In [1]:
import loaders.book_content_loader as bcl

In [2]:
res_search = bcl.search_books("Frankenstein Mary Shelley")

In [3]:
url_archive = bcl.get_book_archive_page(res_search)

In [4]:
book_text = bcl.fetch_book_text(url_archive)

Book found with length: 529209 characters
ustic 
guardians  to  yield  their  charge  to  her.  They  were  fond  of 
the  sweet  orphan.  Her  presence  had  seemed  a  blessing  to 
them ;  but  it  would  be  unfair  to  her  to  keep  her


## RAG with HuggingFace

In [1]:
from langgraph.graph import StateGraph
from transformers import pipeline

c:\Users\NicolasCOTONI\anaconda3\envs\finetune_llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from langchain_huggingface import HuggingFaceEndpoint

model = HuggingFaceEndpoint(
    repo_id="HuggingFaceTB/SmolLM2-135M",
    # max_length=128,
    temperature=0.1,
    # huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"]
)


In [ ]:
# llm_pipeline = pipeline("text-generation", model="HuggingFaceTB/SmolLM2-135M")

c:\Users\NicolasCOTONI\anaconda3\envs\finetune_llm\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\NicolasCOTONI\.cache\huggingface\hub\models--HuggingFaceTB--SmolLM2-135M. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


In [7]:
# Define your state type with relevant keys
class State(dict):
    pass

def llm_node(state: State):
    user_input = state.get("input")
    response = llm_pipeline(user_input, max_length=50)[0]["generated_text"]
    return {"output": response}

graph_builder = StateGraph(State)

# Add node to the graph for LLM inference
graph_builder.add_node("llm_inference", llm_node)

# Set entry and finish points
graph_builder.set_entry_point("llm_inference")
graph_builder.set_finish_point("llm_inference")

# Compile the graph
graph = graph_builder.compile()

# Run the agent
result = graph.invoke({"input": "Who is John in chapter 3?"})
print(result["output"])

TypeError: can only concatenate str (not "NoneType") to str

In [1]:
from rag_system import LocalRAGSystem


c:\Users\NicolasCOTONI\anaconda3\envs\read_recall\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# from rag_system import LocalRAGSystem

# Create a sample document
import os
if not os.path.exists("documents"):
    os.makedirs("documents")

with open("documents/sample.txt", "w") as f:
    f.write("""
Paris is the capital and most populous city of France. It is located in the north-central part of the country.
The Seine river runs through the city. Paris is known for landmarks like the Eiffel Tower, the Louvre Museum,
and Notre-Dame Cathedral. It's often referred to as the "City of Light" or the "City of Love".

Berlin is the capital and largest city of Germany. It is located in northeastern Germany and has a population
of approximately 3.7 million. Berlin is known for its cultural scene, art galleries, and historical landmarks
like the Brandenburg Gate and the Berlin Wall.
""")

# Initialize the RAG system
rag = LocalRAGSystem("documents")
rag.initialize()

# Example query
result = rag.query("What is Paris known for?")
print("\nQuestion: What is Paris known for?")
print("Answer:", result["result"])
print("\nSource documents:")
for i, doc in enumerate(result["source_documents"]):
    print(f"Source {i+1}:", doc.page_content[:100], "...")

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


Loaded 1 document chunks


c:\Users\NicolasCOTONI\Work\ReadRecall\src\rag_system.py:47: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


Vector store created successfully
Loading model: HuggingFaceTB/SmolLM2-135M


Device set to use cpu
c:\Users\NicolasCOTONI\Work\ReadRecall\src\rag_system.py:114: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = self.qa_chain({"query": question})
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


RAG system initialized successfully!

Question: What is Paris known for?
Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Paris is the capital and most populous city of France. It is located in the north-central part of the country. The Seine river runs through the city. Paris is known for landmarks like the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. It's often referred to as the "City of Light" or the "City of Love".

Berlin is the capital and largest city of Germany. It is located in northeastern Germany and has a population of approximately 3.7 million. Berlin is known for its cultural scene, art galleries, and historical landmarks like the Brandenburg Gate and the Berlin Wall.

Question: What is Paris known for?
Helpful Answer: The streets of Paris are lined with famous landmarks such as the Eiffel Tower, the Louvre Museum, the Eiffel Tower, the

In [5]:
print(result.keys())

dict_keys(['query', 'result', 'source_documents'])
